## Convergence Benchmark

In [ ]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit.quantum_info import random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [54]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n
    spam_target = SPAM(d=d)

    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = spam_target.n
    inputs_spam, _ = generate_pauliInput_circuits(n, N_spam)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, noise=0):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm)
    
    return inputs_map, targets_map


def povm_fidelity(spam_a, spam_b):
    d = spam_a.d
    povm_a, povm_b = spam_a.povm, spam_b.povm
    ab = tf.matmul(povm_a, povm_b)
    ab_sqrt = tf.linalg.sqrtm(ab)
    fidelity = tf.math.reduce_sum(tf.linalg.trace(ab_sqrt))/d
    return fidelity


def model_pipeline(channel_target, noise, N_map=2000, rank = None):
    # Make Benchmark
    #################################################################################
    d = channel_target.d
    if rank is None:
        rank = d**2
    
    spam_target = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))

    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 300,
                        verbose = False,
                        )

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = rank,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = 4000,
                N = 500,
                )
    #################################################################################

    return model

## Retrieve SPAM and Map

In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 1
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list1 = []
for i in tqdm(range(10)):
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=None)
    model_list1.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0005139951535039114


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.5193956138541608
None -0.8634358047036302
None -0.9313798626698676
None -0.948363146717546
None -0.9555418138339442
None -0.9592574861373034
None -0.9614230951036297
None -0.962789981505742
None -0.9637046230281023
None -0.9643451882088475
None -0.964810627835349
None -0.9651591589043457
None -0.965426515368078
None -0.9656355303941498
None -0.9658013762210109
None -0.9659344422463557
None -0.9660421665658703
None -0.9661299876815865
None -0.9662019872798442
None -0.9662613111549242
None -0.9663103729245381
None -0.9663510999122882
None -0.9663849521661732
None -0.966413118698981
None -0.9664365487489663
None -0.9664559970446654
None -0.966472092003682
None -0.9664853338056698
None -0.9664961576170029
None -0.9665049563554541
None -0.9665120260194716
None -0.966517651369995
None -0.9665220976034615
None -0.9665255393685107
None -0.9665281732807884
None -0.9665301726250369
None -0.9665316515555149
None -0.9665327303137539
None -0.9665335175042248
None -0.966534050302502
None -0.

...

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0004958215876330925


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.3634836030581072
None -0.8661215303505551
None -0.9395358040033301
None -0.960634975134774
None -0.9688919780600375
None -0.9727818978017435
None -0.9747964979067941
None -0.9758768845877822
None -0.9764534298578402
None -0.9767509618816589
None -0.9768961150803366
None -0.9769617144299413
None -0.976988488143053
None -0.9769978588415132
None -0.9770002955762357
None -0.9770003627111925
None -0.9769998868401267
None -0.9769994356903725
None -0.9769991223925848
None -0.9769989504890665
None -0.976998855717972
None -0.9769988201025073
None -0.9769987828038962
None -0.976998772800658
None -0.9769987794780559
None -0.9769987690837073
None -0.9769987768148825
None -0.976998778392898
None -0.976998779974031
None -0.9769987870607845
None -0.9769987822079435
None -0.9769987804968566
None -0.9769987836878847
None -0.9769987777416068
None -0.9769987756045341
None -0.976998778907639
None -0.9769987809456641
None -0.9769987790980574
None -0.976998771626097
None -0.976998778509048
None -0.9

  0%|          | 0/2000 [00:00<?, ?it/s]

In [67]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list2 = []
for i in tqdm(range(10)):
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=2000)
    model_list2.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0035128648624128196


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.1611729525969062
None -0.6575100759154726
None -0.8525071401954677
None -0.9072456800853229
None -0.9271733260152346
None -0.9358673137075305
None -0.9402820756821223
None -0.9428164857071687
None -0.9444063953913061
None -0.9454663703215939
None -0.9462021953102832
None -0.9467271486782539
None -0.9471089867620782
None -0.947390865960067
None -0.9476015100138695
None -0.9477606271410091
None -0.947882069784073
None -0.9479758162835594
None -0.9480490039140158
None -0.9481068513475994
None -0.9481531488055991
None -0.9481906552573741
None -0.9482214049646596
None -0.9482468939428473
None -0.9482682129789461
None -0.9482862137819864
None -0.9483015211453982
None -0.9483146201656041
None -0.9483259064950605
None -0.9483356539298933
None -0.9483441093694921
None -0.9483514610131139
None -0.948357858608343
None -0.9483634262172961
None -0.9483682640432156
None -0.948372471038357
None -0.9483761104250641
None -0.9483792466312538
None -0.9483819474023468
None -0.9483842475114288
None

...

None -0.1675582976821294
None -0.6130717664974071
None -0.8142805981481465
None -0.8879476357108262
None -0.921376692882244
None -0.9387262025543945
None -0.9486787404055921
None -0.9548407877957276
None -0.9588880278145889
None -0.9616706260477329
None -0.9636542860393003
None -0.9651104190533938
None -0.9662049859126828
None -0.9670437177699865
None -0.9676963565664024
None -0.9682104984294111
None -0.968619578521158
None -0.9689476988585477
None -0.9692127031307068
None -0.9694279813876249
None -0.9696038167495313
None -0.969748131364336
None -0.9698671139658359
None -0.9699656327558028
None -0.9700475248321102
None -0.9701158433111172
None -0.970173032968259
None -0.9702210293802538
None -0.9702614346890921
None -0.9702955190857279
None -0.97032430918124
None -0.9703486574924692
None -0.9703692549825561
None -0.9703866909258173
None -0.9704014068421656
None -0.9704138105395282
None -0.9704242243322008
None -0.9704329222746637
None -0.9704401426732826
None -0.9704460982459043
None -

  0%|          | 0/2000 [00:00<?, ?it/s]

0.002776469310265193


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.19653637392309092
None -0.6665344973004917
None -0.8404738782486626
None -0.902447034508727
None -0.929837297893566
None -0.9435565750314466
None -0.9510505135966746
None -0.9554512767700977
None -0.9582039297111369
None -0.9600170960663242
None -0.9612598669138988
None -0.962137077275643
None -0.9627697089615642
None -0.9632334214284232
None -0.9635776611998788
None -0.9638359174798685
None -0.9640314234100161
None -0.9641806040678425
None -0.9642952958171158
None -0.9643840311380183
None -0.9644530711105401
None -0.9645070401979627
None -0.9645493614594515
None -0.9645826239247061
None -0.9646087716067133
None -0.9646292825232066
None -0.96464532592199
None -0.9646578134000082
None -0.9646674685444484
None -0.9646748741635611
None -0.9646804789266604
None -0.9646846934840032
None -0.9646878053450613
None -0.9646900743591423
None -0.9646916937091692
None -0.9646928408799939
None -0.9646936267516961
None -0.9646941561262196
None -0.9646945092031867
None -0.9646947349540927
None

In [66]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list3 = []
for i in tqdm(range(10)):
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=5000)
    model_list3.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.006012399588972417


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.10065928648094982
None -0.42957671218682547
None -0.6386511206620739
None -0.7513222718727653
None -0.8197354112952818
None -0.8622896022869961
None -0.8895552431780325
None -0.9079748165335095
None -0.920437482248729
None -0.9291298070402664
None -0.9351789847774139
None -0.9396668039784511
None -0.9429203456951449
None -0.945221277129166
None -0.9471275350493173
None -0.9485615772152421
None -0.9497661830010089
None -0.950715783234997
None -0.9512390776696525
None -0.9518391103310591
None -0.9521789660523379
None -0.9527045972079315
None -0.9528393458202079
None -0.9531952881743172
None -0.9534415870329657
None -0.9536929397724058
None -0.9538117280019999
None -0.9540697358279531
None -0.9542203870488226
None -0.9543639580733457
None -0.9543354721597302
None -0.9542386231652625
None -0.9542559584314313
None -0.9545738554915794
None -0.95440881493173
None -0.9546180725839454
None -0.9546947953182057
None -0.9546812422404442
None -0.9546191967528369
None -0.9548100661627871
Non

...

None -0.09354005178893231
None -0.41050718089263755
None -0.633251418668288
None -0.7457676928647359
None -0.8147378220747934
None -0.8585371987840356
None -0.8870535603688846
None -0.9063478528902855
None -0.9196210929983296
None -0.9288676964325313
None -0.935357746425808
None -0.9400540882476953
None -0.9434959066243139
None -0.9459850002520985
None -0.9478991358431603
None -0.9494713043888269
None -0.9505211125167975
None -0.9514595216844247
None -0.9521376711382896
None -0.952920103619031
None -0.9533855635989965
None -0.953720874075874
None -0.9542181484959584
None -0.9543388575194688
None -0.9544256511648497
None -0.9545799987043078
None -0.9550555419353342
None -0.9550860684634965
None -0.9552265707888595
None -0.9551685464177232
None -0.9553892857141378
None -0.9553069149601727
None -0.9554873804023871
None -0.9553120985428016
None -0.9553237694689748
None -0.9555213048944883
None -0.9556403823322694
None -0.955561943986313
None -0.9560124758145704
None -0.9557610007047729
Non

  0%|          | 0/2000 [00:00<?, ?it/s]

0.00664913501531487


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.09281837948759258
None -0.41709000623751236
None -0.6354517127944724
None -0.7497651469085891
None -0.8187425949252706
None -0.8612472105150535
None -0.8884944140775286
None -0.9065746699864112
None -0.9186284700954495
None -0.9269746318039118
None -0.9328918228315419
None -0.9369273729416582
None -0.9398301164115942
None -0.942104800237913
None -0.9438490081834672
None -0.9451945890831375
None -0.9461824042962974
None -0.9469250175266518
None -0.9478246506524359
None -0.9484741094991219
None -0.948540918628314
None -0.9490518152494203
None -0.9491609621755487
None -0.9493298964849941
None -0.9497661596277931
None -0.950091860983549
None -0.9500819958908734
None -0.9501109042515894
None -0.9502935359607514
None -0.9505799007937913
None -0.9506055862370415
None -0.9508302455472234
None -0.9505014383235053
None -0.950620399360571
None -0.950556687042137
None -0.9507117378911256
None -0.9504662353039711
None -0.9509182580673501
None -0.9508410762307461
None -0.9510543738040729
Non

In [ ]:
fidelity_oneQubits = [model.logger.loss_val_list for model in model_list1]
fidelity_twoQubits = [model.logger.loss_val_list for model in model_list2]
fidelity_threeQubits = [model.logger.loss_val_list for model in model_list3]

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=300, facecolor="w", edgecolor="k")

plt.plot(-np.array(fidelity_oneQubits).T,"g-")
plt.plot(-np.array(fidelity_twoQubits).T,"b-")
plt.plot(-np.array(fidelity_threeQubits).T,"r-")
#plt.yscale("log")
plt.ylim([0.9, 1.05])
plt.grid()
plt.show()